In [1]:
import math
import os
import time
from datetime import datetime
from pathlib import Path
from glob import glob

import pickle as pkl
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import loadmat
from PIL import Image

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import datasets, transforms, utils
from PIL import Image
import numpy as np
from torchvision.utils import save_image


%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [2]:
import torch.nn as nn
import torch.nn.functional as F

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
def conv(in_channels, out_channels, kernel_size=4, stride=2, padding=1, batch_norm=True):
    layers = []
    conv_layer = nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
    layers.append(conv_layer)
    
    if batch_norm:
        bn = nn.BatchNorm2d(out_channels)
        layers.append(bn)
        
    return nn.Sequential(*layers)

In [5]:
# class Discriminator(nn.Module):
#     def __init__(self, y_size, conv_dim=128):
#         super(Discriminator, self).__init__()
        
#         self.conv_dim = conv_dim
        
#         # Convolutional layers
#         self.conv1 = nn.Conv2d(3, conv_dim, 4, 2, 1)
#         self.conv2 = nn.Conv2d(conv_dim, conv_dim*2, 4, 2, 1)
#         self.conv3 = nn.Conv2d(conv_dim*2, conv_dim*4, 4, 2, 1)
#         self.conv4 = nn.Conv2d(conv_dim*4, conv_dim*8, 4, 2, 1)
#         self.conv5 = nn.Conv2d(conv_dim*8 + y_size, 1, 4, 1, 0)
        
#     def forward(self, x, y):
#         x = nn.LeakyReLU(0.2)(self.conv1(x))
#         x = nn.LeakyReLU(0.2)(self.conv2(x))
#         x = nn.LeakyReLU(0.2)(self.conv3(x))
#         x = nn.LeakyReLU(0.2)(self.conv4(x))
        
#         # Concatenate the condition y to the features
#         y = y.view(-1, y.size()[-1], 1, 1)
#         y = y.expand(-1, -1, x.size(2), x.size(3))
#         x = torch.cat([x, y], dim=1)
        
#         x = self.conv5(x)
        
#         return x

In [6]:
class Discriminator(nn.Module):
    
    def __init__(self, y_size, conv_dim=64):

        super(Discriminator, self).__init__()
        self.conv_dim = conv_dim
        self.y_size = y_size
        self.conv1 = conv(3, conv_dim, 4, batch_norm=False)
        self.conv2 = conv(conv_dim+y_size, conv_dim * 2, 4)
        self.conv3 = conv(conv_dim*2, conv_dim*4, 4)
        self.conv4 = conv(conv_dim*4, conv_dim*8, 4)
        self.conv5 = conv(conv_dim*8, 1, 4, 1, 0, batch_norm=False)
            
    def forward(self, x, y):

        x = F.relu(self.conv1(x))
        y = y.view(-1,y.size()[-1],1,1)
        y = y.expand(-1,-1,x.size()[-2], x.size()[-1])
        x = torch.cat([x, y], 1)
        x = F.relu(self.conv2(x))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        x = self.conv5(x)
        
        return x

In [7]:
def deconv(in_channels, out_channels, kernel_size=4, stride=2, padding=1, batch_norm=True):
    
    layers = []
    t_conv = nn.ConvTranspose2d(in_channels, out_channels, kernel_size, stride, padding, bias=False)
    layers.append(t_conv)
    
    if batch_norm:
        layers.append(nn.BatchNorm2d(out_channels))
        
    return nn.Sequential(*layers)

In [8]:
# class Generator(nn.Module):
#     def __init__(self, z_size, y_size, conv_dim=128):
#         super(Generator, self).__init__()
        
#         self.conv_dim = conv_dim
        
#         # Transposed convolutional layers
#         self.t_conv1 = nn.ConvTranspose2d(z_size + y_size, conv_dim*8, 4, 1, 0)
#         self.t_conv2 = nn.ConvTranspose2d(conv_dim*8, conv_dim*4, 4, 2, 1)
#         self.t_conv3 = nn.ConvTranspose2d(conv_dim*4, conv_dim*2, 4, 2, 1)
#         self.t_conv4 = nn.ConvTranspose2d(conv_dim*2, conv_dim, 4, 2, 1)
#         self.t_conv5 = nn.ConvTranspose2d(conv_dim, 3, 4, 2, 1)
        
#     def forward(self, z, y):
#         x = torch.cat([z, y], dim=1)
#         x = x.view(-1, x.size()[-1], 1, 1)
        
#         x = nn.ReLU()(self.t_conv1(x))
#         x = nn.ReLU()(self.t_conv2(x))
#         x = nn.ReLU()(self.t_conv3(x))
#         x = nn.ReLU()(self.t_conv4(x))
#         x = torch.tanh(self.t_conv5(x))
        
#         return x

In [9]:
class Generator(nn.Module):
    
    def __init__(self, z_size, y_size, conv_dim=64):

        super(Generator, self).__init__()
        
        self.conv_dim = conv_dim
        
        self.t_conv1 = deconv(z_size+y_size, conv_dim*8, 4, 1, 0)
        self.t_conv2 = deconv(conv_dim*8, conv_dim*4, 4)
        self.t_conv3 = deconv(conv_dim*4, conv_dim*2, 4)
        self.t_conv4 = deconv(conv_dim*2, conv_dim, 4)
        self.t_conv5 = deconv(conv_dim, 3, 4, batch_norm=False)
        
    def forward(self, z, y):

        x = torch.cat([z, y], dim=1)
        x = x.view(-1, x.size()[-1], 1, 1)
        x = F.relu(self.t_conv1(x))
        x = F.relu(self.t_conv2(x))
        x = F.relu(self.t_conv3(x))
        x = F.relu(self.t_conv4(x))
        x = self.t_conv5(x)
        x = torch.tanh(x)
        
        return x

In [10]:
# # Generator
# class Generator(nn.Module):
#     def __init__(self, z_dim, y_dim, image_size, num_channels):
#         super(Generator, self).__init__()
        
#         self.z_dim = z_dim
#         self.y_dim = y_dim
#         self.image_size = image_size
#         self.num_channels = num_channels
        
#         self.fc = nn.Sequential(
#             nn.Linear(z_dim + y_dim, 256 * 8 * 8),
#             nn.ReLU()
#         )
        
#         self.deconv_layers = nn.Sequential(
#             nn.ConvTranspose2d(256, 128, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(128),
#             nn.ReLU(),
            
#             nn.ConvTranspose2d(128, 64, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(64),
#             nn.ReLU(),
            
#             nn.ConvTranspose2d(64, num_channels, kernel_size=4, stride=2, padding=1),
#             nn.Tanh()
#         )

#     def forward(self, z, y):
#         zy = torch.cat([z, y], dim=1)
#         x = self.fc(zy)
#         x = x.view(x.size(0), 256, 8, 8)
#         x = self.deconv_layers(x)
#         return x

# # Discriminator
# class Discriminator(nn.Module):
#     def __init__(self, image_size, y_dim, num_channels):
#         super(Discriminator, self).__init__()
        
#         self.image_size = image_size
#         self.y_dim = y_dim
#         self.num_channels = num_channels
        
#         self.conv_layers = nn.Sequential(
#             nn.Conv2d(num_channels + y_dim, 64, kernel_size=4, stride=2, padding=1),
#             nn.LeakyReLU(0.2),
            
#             nn.Conv2d(64, 128, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(128),
#             nn.LeakyReLU(0.2),
            
#             nn.Conv2d(128, 256, kernel_size=4, stride=2, padding=1),
#             nn.BatchNorm2d(256),
#             nn.LeakyReLU(0.2)
#         )
        
#         self.fc = nn.Sequential(
#             nn.Linear(256 * 8 * 8, 1),
#             nn.Sigmoid()
#         )

#     def forward(self, x, y):
#         y = y.view(-1, self.y_dim, 1, 1)
#         y = y.expand(-1, -1, self.image_size, self.image_size)
#         x = torch.cat([x, y], dim=1)
#         x = self.conv_layers(x)
#         x = x.view(x.size(0), -1)
#         x = self.fc(x)
#         return x

In [11]:
conv_dim = 128
z_size = 100
y_size = 6 

D = Discriminator(y_size, conv_dim)
G = Generator(z_size, y_size, conv_dim)

print(D)
print()
print(G)

Discriminator(
  (conv1): Sequential(
    (0): Conv2d(3, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
  )
  (conv2): Sequential(
    (0): Conv2d(134, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv3): Sequential(
    (0): Conv2d(256, 512, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv4): Sequential(
    (0): Conv2d(512, 1024, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (conv5): Sequential(
    (0): Conv2d(1024, 1, kernel_size=(4, 4), stride=(1, 1), bias=False)
  )
)

Generator(
  (t_conv1): Sequential(
    (0): ConvTranspose2d(106, 1024, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(1024, eps

In [12]:
# # Initialize the generator and discriminator
# z_dim = 100
# y_dim = 6
# image_size = 64
# num_channels = 3

# G = Generator(z_dim, y_dim, image_size, num_channels)
# D = Discriminator(image_size, y_dim, num_channels)

# print(D)
# print()
# print(G)

In [13]:
bins = [18, 29, 39, 49, 59]
root_dir = "D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN"

In [14]:
G.load_state_dict(torch.load(os.path.join(root_dir,'GAN_mod_1_G.pth'), map_location=torch.device(device)))
D.load_state_dict(torch.load(os.path.join(root_dir,'GAN_mod_1_D.pth'), map_location=torch.device(device)))

<All keys matched successfully>

In [15]:
!cd

D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main


In [16]:
# Move your model to the specified device
G = G.to(device)

num_samples = 16
z = torch.randn(num_samples, z_size, device=device)
ages = torch.zeros(num_samples, len(bins) + 1, device=device).scatter_(
    1, torch.tensor(np.random.randint(len(bins), size=num_samples).reshape(-1, 1), dtype=torch.long, device=device), 1
)

fake_images = G(z, ages).detach().cpu()

os.makedirs(os.path.join(root_dir, 'samples'), exist_ok=True)
print("done created")

for i in range(num_samples):
    age_label = np.argmax(ages[i].cpu().numpy())
    filename = os.path.join(root_dir, 'samples', 'sample_{}_age_{}.png'.format(i, age_label))
    save_image(fake_images[i], filename)

done created


In [17]:
for i in range(num_samples):
    age_label = np.argmax(ages[i].cpu().numpy())
    filename = os.path.abspath(os.path.join(root_dir, 'samples', 'sample_{}_age_{}.png'.format(i, age_label)))
    save_image(fake_images[i], filename)
    print("Saved sample image:", filename)

Saved sample image: D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN\samples\sample_0_age_3.png
Saved sample image: D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN\samples\sample_1_age_0.png
Saved sample image: D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN\samples\sample_2_age_0.png
Saved sample image: D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN\samples\sample_3_age_0.png
Saved sample image: D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN\samples\sample_4_age_2.png
Saved sample image: D:\study\Project\TARP project\MissingChildIdentification-main\MissingChildIdentification-main\content\Age-cGAN\samples\sample_5_age_1.png
Saved sample image: D:\study\Project\TARP project\Mi

In [18]:
# # Set the model to evaluation mode
# G.eval()

# # Evaluate the model on the test set
# test_loss = 0
# mae = 0
# num_batches = 0
# with torch.no_grad():
#     for batch in test_loader:
#         # Get the batch size
#         batch_size = batch['image'].size(0)
        
#         # Scale the images and one-hot encode the ages
#         real_images = scale(batch['image'])
#         ages = one_hot(batch['age'], bins)
        
#         # Move the data to the device
#         real_images = real_images.to(device)
#         ages = ages.to(device)
        
#         # Generate fake images and calculate the loss
#         z = np.random.uniform(-1, 1, size=(batch_size, z_size))
#         z = torch.from_numpy(z).float().to(device)
#         fake_images = G(z, ages)
#         D_fake = D(fake_images, ages)
#         g_loss = real_loss(D_fake)
#         test_loss += g_loss.item() * batch_size
        
#         # Calculate the predicted ages and the MAE
#         predicted_ages = unnormalize_age(predict_age(fake_images, bins))
#         true_ages = batch['age']
#         mae += torch.sum(torch.abs(predicted_ages - true_ages)).item()
#         num_batches += 1

# # Calculate the average test loss and MAE
# test_loss /= len(test_loader.dataset)
# mae /= len(test_loader.dataset)

# # Print the results
# print('Test Loss: {:.4f}, MAE: {:.2f}'.format(test_loss, mae))